# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [2]:
import numpy as np

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2024-11-28 14:02:07.824620: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-28 14:02:14.052097: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWGKGDM/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWGKGDM/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2024-11-28 14:02:42.004212: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure: 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [4]:
from gensim.models import Word2Vec

In [5]:
word2vec = Word2Vec(sentences=X_train,
                    vector_size=50,
                    window=5,
                    min_count=1) 

Let's reuse the functions of the previous exercise to convert your training and test data into something you can feed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [6]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [7]:
X_train_pad, X_test_pad

(array([[[-2.06619596e+00,  1.88450545e-01,  2.66865164e-01, ...,
           1.03391063e+00,  2.89795429e-01,  3.20043743e-01],
         [-7.64735937e-02, -8.60516205e-02, -1.01234066e+00, ...,
          -6.41835527e-03,  3.87255549e-01, -9.07036424e-01],
         [-1.14496028e+00,  2.07276970e-01,  8.98535967e-01, ...,
          -5.57932317e-01,  3.19506764e-01,  2.39436555e+00],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],
 
        [[-3.52093124e+00, -2.15976164e-02, -2.22630143e+00, ...,
           8.86069238e-01,  1.36235285e+00, -1.53892159e+00],
         [-5.53387284e-01, -2.41613436e+00,  1.42338693e+00, ...,
          -1.88252270

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad`:
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from finding errors too late and from letting them propagate through the entire notebook.

In [8]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you are doing something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [10]:
from collections import Counter

In [11]:
label_counts = Counter(y_train)
label_counts

Counter({0: 1265, 1: 1235})

In [15]:
len(y_train)

2500

In [14]:
baseline_accuracy = max(label_counts.values()) / len(y_train)
baseline_accuracy

0.506

La etiqueta más frecuente en y_train representa el 50.6% del total de las muestras. Hay que superar este valor con el modelo para demostrar que está aprendiendo patrones significativos en los datos.

# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense

In [17]:
model = Sequential([
    # Masking layer
    Masking(mask_value=0.0, input_shape=(None, 50)),
    LSTM(20, activation='tanh'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (None, None, 50)          0         
                                                                 
 lstm (LSTM)                 (None, 20)                5680      
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 5,901
Trainable params: 5,901
Non-trainable params: 0
_________________________________________________________________


❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy will greatly depend on your training corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

In [20]:
es = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

In [21]:
history = model.fit(
    X_train_pad, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=32,
    callbacks=[es]
)

Epoch 1/20
63/63 [==============================] - 8s 93ms/step - loss: 0.6954 - accuracy: 0.4855 - val_loss: 0.6952 - val_accuracy: 0.4820
Epoch 2/20
63/63 [==============================] - 4s 60ms/step - loss: 0.6873 - accuracy: 0.5420 - val_loss: 0.6826 - val_accuracy: 0.5580
Epoch 3/20
63/63 [==============================] - 4s 61ms/step - loss: 0.6803 - accuracy: 0.5685 - val_loss: 0.6716 - val_accuracy: 0.5880
Epoch 4/20
63/63 [==============================] - 4s 63ms/step - loss: 0.6722 - accuracy: 0.5825 - val_loss: 0.6541 - val_accuracy: 0.6300
Epoch 5/20
63/63 [==============================] - 4s 61ms/step - loss: 0.6661 - accuracy: 0.5875 - val_loss: 0.6509 - val_accuracy: 0.6380
Epoch 6/20
63/63 [==============================] - 4s 59ms/step - loss: 0.6602 - accuracy: 0.6060 - val_loss: 0.6463 - val_accuracy: 0.6320
Epoch 7/20
63/63 [==============================] - 4s 61ms/step - loss: 0.6565 - accuracy: 0.6125 - val_loss: 0.6503 - val_accuracy: 0.6320
Epoch 8/20
63

❓ **Question** ❓ Evaluate your model on the test set

In [22]:
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test)
test_loss, test_accuracy

79/79 [==============================] - 1s 13ms/step - loss: 0.6486 - accuracy: 0.6452


(0.6486198306083679, 0.6452000141143799)

# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. Let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that others have learned? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this: 

In [23]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is smaller (65 MB).

</details>

In [24]:
word2vec_transfer = api.load("glove-wiki-gigaword-50")

[===============================-------------------] 63.5% 41.9/66.0MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 66.0/66.0MB downloaded


❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [25]:
len(word2vec_transfer.key_to_index)

400000

In [26]:
word2vec_transfer.vector_size

50

❓ Let's embed `X_train` and `X_test`, same as in the first question where we provided the functions to do so! (There is a slight difference in the `embed_sentence_with_TF` function that we will not dig into)

In [27]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [28]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)
X_train_pad_2.shape, X_test_pad_2.shape

((2500, 200, 50), (2500, 200, 50))

❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training here could take some time. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [30]:
model = Sequential([
    Masking(mask_value=0.0, input_shape=(200, 50)),  # Longitud máxima: 200, Embedding dim: 50 como el shape!
    LSTM(20, activation='tanh'),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_pad_2, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[es]
)

Epoch 1/10
63/63 [==============================] - 8s 78ms/step - loss: 0.6884 - accuracy: 0.5370 - val_loss: 0.6772 - val_accuracy: 0.5740
Epoch 2/10
63/63 [==============================] - 4s 69ms/step - loss: 0.6662 - accuracy: 0.5995 - val_loss: 0.6562 - val_accuracy: 0.6240
Epoch 3/10
63/63 [==============================] - 6s 95ms/step - loss: 0.6357 - accuracy: 0.6375 - val_loss: 0.6290 - val_accuracy: 0.6320
Epoch 4/10
63/63 [==============================] - 4s 64ms/step - loss: 0.6097 - accuracy: 0.6690 - val_loss: 0.6116 - val_accuracy: 0.6500
Epoch 5/10
63/63 [==============================] - 4s 64ms/step - loss: 0.5882 - accuracy: 0.6905 - val_loss: 0.5925 - val_accuracy: 0.7000
Epoch 6/10
63/63 [==============================] - 4s 71ms/step - loss: 0.5610 - accuracy: 0.7155 - val_loss: 0.6136 - val_accuracy: 0.6800
Epoch 7/10
63/63 [==============================] - 4s 61ms/step - loss: 0.5540 - accuracy: 0.7215 - val_loss: 0.5888 - val_accuracy: 0.6740
Epoch 8/10
63

In [31]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 73.160%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarded words that were not present more than a given number of times in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously

### El modelo clasificó correctamente el 73.16% de las muestras del conjunto de prueba como 0 o 1. Esto indica que el modelo es razonablemente bueno para analisis de sentimientos, podemos jugar con mas parametros como: 

LSTM, mas capas
 
batch size u otros optimizadores como adam
 
normalizar el texto
 
utilizar otros modelos preentrenados